In [10]:
!pip install weaviate-client langchain tiktoken pydf rapidocr-onnxruntime

In [14]:
WEAVIATE_API_KEY = "szQKHcXpFvgPvdqsMpk3uC0siVLWBQffIjq6"
WEAVIATE_CLUSTER_URL ="https://ozplqqvrsym2rlayd3l7a.c0.us-east1.gcp.weaviate.cloud"

In [15]:
from langchain.vectorstores import Weaviate
import weaviate

WEAVIATE_API_KEY = WEAVIATE_API_KEY
WEAVIATE_CLUSTER_URL = WEAVIATE_CLUSTER_URL

client = weaviate.Client(
    url=WEAVIATE_CLUSTER_URL,
    auth_client_secret = weaviate.AuthApiKey(WEAVIATE_API_KEY)

)

<ipython-input-15-17d45377a97f>:7: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [18]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 7.2 MB/s eta 0:00:00


In [5]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00


In [22]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name = embedding_model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 7.3 MB/s eta 0:00:00


In [13]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/rag.pdf", extract_images=True)
pages = loader.load()

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

In [27]:
print(docs)

[Document(metadata={'source': '/content/rag.pdf', 'page': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research;‡University College London;⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate knowl-\nedge is still limited, and hence on knowledge-intensive tasks, their performance\nlags behind task-speciﬁc architectures. Additionally, providing provenance for their\ndecisions and updating their world knowledge remain open research problems. Pre-\ntrained models with a differentiable access mechanism to expli

In [28]:
vector_db = Weaviate.from_documents(
    documents = docs,
    embedding = embeddings,
    client = client,
    by_text = False,
)

In [30]:
vector_db.similarity_search(query = "what is rag?", k= 3)

[Document(metadata={'page': 17, 'source': '/content/rag.pdf'}, page_content='documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document\nmechanisms may not be necessary for RAG.\nG Parameters\nOur RAG models contain the trainable parameters for the BERT-base query and document encoder of\nDPR, with 110M parameters each (although we do not train the document encoder ourselves) and\n406M trainable parameters from BART-large, 406M parameters, making a total of 626M trainable\n18'),
 Document(metadata={'page': 5, 'source': '/content/rag.pdf'}, page_content='cases for NQ, where an extractive model would score 0%.\n4.2 Abstractive Question Answering\nAs shown in Table 2, RAG-Sequence outperforms BART on Open MS-MARCO NLG by 2.6 Bleu\npoints and 2.6 Rouge-L points. RAG approaches state-of-the-art model performance, which is\nimpressive given that (i) those models access gold passages with speciﬁc information required to\ngenerate the reference answer 

In [32]:
print(
    vector_db.similarity_search(query = "what is rag?", k= 3)[0].page_content)

documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document
mechanisms may not be necessary for RAG.
G Parameters
Our RAG models contain the trainable parameters for the BERT-base query and document encoder of
DPR, with 110M parameters each (although we do not train the document encoder ourselves) and
406M trainable parameters from BART-large, 406M parameters, making a total of 626M trainable
18


In [33]:
from langchain.prompts import ChatPromptTemplate\

template = """


you are an assistant for question and answering tasks.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer
Question: {question}
=========
{context}
=========
Answer in Markdown:
"""
prompt = ChatPromptTemplate.from_template(template)



In [34]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n\n\nyou are an assistant for question and answering tasks.\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer\nQuestion: {question}\n=========\n{context}\n=========\nAnswer in Markdown:\n"), additional_kwargs={})])

In [35]:
from langchain import HuggingFaceHub

In [37]:
from google.colab import userdata
huggingface_api_token = userdata.get('HF_TOKEN')

In [39]:
model = HuggingFaceHub(
    huggingfacehub_api_token =huggingface_api_token ,
    repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs = {"temperature": 0.5, "max_length": 180}
)

In [40]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [41]:
retriever = vector_db.as_retriever()

In [42]:
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | promptu
    | model
    | StrOutputParser()
)

In [47]:
print(rag_chain.invoke(" rag vs finetuning"))

Human: 


you are an assistant for question and answering tasks.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer
Question:  rag vs finetuning
[Document(metadata={'page': 7, 'source': '/content/rag.pdf'}, page_content='documents. Figure 3 (right) shows that retrieving more documents leads to higher Rouge-L for\nRAG-Token at the expense of Bleu-1, but the effect is less pronounced for RAG-Sequence.\n10 20 30 40 50\nKR e t r i e v e dD o c s394041424344NQ Exact MatchRAG-Tok\nRAG-Seq\n10 20 30 40 50\nKR e t r i e v e dD o c s4050607080NQ Answer Recall @ KRAG-Tok\nRAG-Seq\nFixed DPR\nBM25\n10 20 30 40 50\nKR e t r i e v e dD o c s4850525456Bleu-1 / Rouge-L scoreRAG-Tok R-L\nRAG-Tok B-1\nRAG-Seq R-L\nRAG-Seq B-1\nFigure 3: Left: NQ performance as more documents are retrieved. Center: Retrieval recall perfor-\nmance in NQ. Right: MS-MARCO Bleu-1 and Rouge-L as more documents are r